## Justin Parsons
MSDS 600 Week 5 Assignment \
2/11/2023

# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

**I chose to use TPOT and not deal the version issues asociated with pycaret**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import timeit 

In [2]:
import pandas as pd

df = pd.read_csv('cleaned_churn_data.csv', index_col='customerID')
df.head()

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
customerID,,,,,,,,
7590-VHVEG,1.0,0,0,3,29.85,29.85,0,29.850000
5575-GNVDE,34.0,1,1,2,56.95,1889.50,0,55.573529
3668-QPYBK,2.0,1,0,2,53.85,108.15,1,54.075000
7795-CFOCW,45.0,0,1,1,42.30,1840.75,0,40.905556
9237-HQITU,2.0,1,0,3,70.70,151.65,1,75.825000


In [3]:
features = df.drop('Churn', axis=1)
targets = df['Churn']

x_train, x_test, y_train, y_test = train_test_split(features, targets, stratify=targets, random_state=42)

In [4]:
%%time
tpot = TPOTClassifier(generations=25, population_size=100, cv=10,random_state=42, scoring='accuracy', verbosity=2, n_jobs=-1)

tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))

Optimization Progress:   0%|          | 0/2600 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7974218794752821

Generation 2 - Current best internal CV score: 0.7989388211032823

Generation 3 - Current best internal CV score: 0.7993151028240821

Generation 4 - Current best internal CV score: 0.7993151028240821

Generation 5 - Current best internal CV score: 0.7993151028240821

Generation 6 - Current best internal CV score: 0.7993151028240821

Generation 7 - Current best internal CV score: 0.7993151028240821

Generation 8 - Current best internal CV score: 0.7998843587099731

Generation 9 - Current best internal CV score: 0.8002617144984819

Generation 10 - Current best internal CV score: 0.8002617144984819

Generation 11 - Current best internal CV score: 0.8002652947241794

Generation 12 - Current best internal CV score: 0.8006415764449789

Generation 13 - Current best internal CV score: 0.8006415764449789

Generation 14 - Current best internal CV score: 0.8006415764449789

Generation 15 - Current best internal CV score: 0.800641

I settled on this model after running quite a few different iterations of TPOT with different combinations of population and generations. I ran with both parameters doubled and accidentaly re-ran and lost it. The benefit was only a CV increase of about 0.1%. I received an error (really, a warning and not an actual error),
    
    (serWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
    warnings.warn) 
    
due to one of the models however, I found documentation online stating it was a known bug for that model. 
  

In [5]:
predictions = tpot.predict(x_test)
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [6]:
# display the actuals and predictions for the test set
print('Predictions for test data set')
print(predictions)
print('Actuals for test data set')
print(y_test)

Predictions for test data set
[0 0 0 ... 0 0 0]
Actuals for test data set
customerID
5343-SGUBI    0
5442-BXVND    0
6434-TTGJP    0
1628-BIZYP    0
0298-XACET    0
             ..
3780-DDGSE    0
4154-AQUGT    1
1116-DXXDF    0
3237-AJGEH    1
6704-UTUKK    0
Name: Churn, Length: 1761, dtype: int64


In [7]:
from sklearn.metrics import accuracy_score
print(f'Accuracy of the TPOT predictions: {accuracy_score(y_test,predictions)}')

Accuracy of the TPOT predictions: 0.7955706984667802


So, it looks like the model is slightly overfit. Everything below here relates to exporting the model and making predictions on a new data set. 

In [8]:
tpot.export('tpot_churn_pipeline.py')

Here I'm displaying the exported code for verification.

In [17]:
from IPython.display import Code

Code('tpot_churn_pipeline.py')

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: 0.8023450478318152
exported_pipeline = make_pipeline(
    PCA(iterated_power=7, svd_solver="randomized"),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=1, max_features=0.35000000000000003, min_samples_leaf=16, min_samples_split=20, n_estimators=100, subsample=0.5)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [23]:
from IPython.display import Code

Code('tpot_churn_pipeline_filled.py')

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
print('Please enter the data filename including file extension. Only .csv files are acceptable.')
file = input()
tpot_data = pd.read_csv(file, index_col='customerID')
features = tpot_data.drop('Churn', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Churn'], random_state=42)

# Average CV score on the training set was: 0.8023450478318152
exported_pipeline = make_pipeline(
    PCA(iterated_power=7, svd_solver="randomized"),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=1, max_features=0.35000000000000003, min_samples_leaf=16, min_samples_split=20, n_estimators=100, subsample=0.5)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
print(results)

In [26]:
%run tpot_churn_pipeline_filled.py

Please enter the data filename including file extension. Only .csv files are acceptable.
new_churn_data.csv


KeyError: "['Churn'] not found in axis"

# Summary

I had to visit the epistasis (epistaxis is a nose bleed so the name through me off at first) website for the tpot readme to get installation instructions. I installed tpot and the extra dependencies and had no issues. 

Cleaned data from previous weeks was read in and split into features and target. The data was further split into test and training sets. TPOTClassifier was run multiple times with various configurations of generations, population size, scoring parameter, and number of folds used to calculate CV of that parameter. A maximum accuracy of approximately 80% was achieved for the model "Gradient Boosting Classifier" which TPOT tested amongst others. The important part of this assignment was that TPOT was able to run our data through multiple model pipelines and choose the best one. With the optimal pipeline there was negligible overfitting. 

Embarassingly, I spent  A LOT of time trying to figure out why the script was not running properly against a data set without a target column (just the new_churn_data.csv set) at all. I went back looked more carefully at your code and realized you were verifying against the same data set. However, I did end up learning a little bit about the input requirements for the model because of this.

I had used github in the past so I created a new repository for this course and committed my python notebook and script to it. 
